## Импорт библиотек

In [1]:
from ultralytics import YOLO
import torch
import cv2
import mediapipe as mp

## Загрузка модели

In [2]:
# веса лучшей модели
model_ = YOLO("/home/pinea/work/YOLO_project/runs/classify/MAIN/weights/best.pt")
# для запуска на cpu или gpu
model_ = model_.to('cuda:0')
# model_ = model_.to('cpu')
print(f"Модель работает на устройстве: {next(model_.parameters()).device}")

Модель работает на устройстве: cuda:0


In [ ]:
# для обнаружения области с лицом на изображении используется MediaPipe
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1744904413.116611   40998 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
def detect_face(frame):
    '''обнаружение лица на изображении/кадре из видео, чтобы получить в качестве изображения область с лицом.'''
    if isinstance(frame, str):
        frame = cv2.imread(frame)
    results = face_detection.process(frame)
    
    if results.detections:
    # получение данных об обнаруженных областях
        for i in results.detections:
            relative_bounding_box = i.location_data.relative_bounding_box
            
            # получение координат области
            w = int(relative_bounding_box.width * frame.shape[1])
            h = int(relative_bounding_box.height * frame.shape[0])
            x = max(0, int(relative_bounding_box.xmin * frame.shape[1]) - int(w * 0.125))  
            y = max(0, int(relative_bounding_box.ymin * frame.shape[0]) - int(h * 0.125))  

            # возвращает обрезанное изображение(область с лицом)
            return frame[y:y+int(h*1.25), x:x+int(w*1.25)]
    else: raise ValueError('лица не обнаружены')

In [ ]:
def prediction_image(image, model):
    """Для обработки фото"""
    result = model.predict(source=detect_face(image), verbose=False)
    result[0].probs.top1
    class_name = result[0].names[result[0].probs.top1]
    print(f"Изображение: {class_name}")
    
prediction_image("test_data/test.jpg", model_)

Изображение: fake


In [ ]:
def prediction_video(video_path, model):
    '''метод для обработки видео'''
    class_counts={'fake' : 0, 'real' : 0}

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise ValueError("Не удалось открыть видеофайл")

    while cap.isOpened():
        ret, frame = cap.read()
        if ret != True:
            break 

        result = model.predict(source=detect_face(frame), verbose=False)
        
        class_name = result[0].names[result[0].probs.top1]

        class_counts[class_name if class_name == 'fake' else 'real'] += 1
    fake_percentage = (class_counts['fake'] / (class_counts['fake'] + class_counts['real'])) * 100
    real_percentage = (class_counts['real'] / (class_counts['fake'] + class_counts['real'])) * 100
    
    print(f"Процент фейковых кадров: {fake_percentage:.2f}%")
    print(f"Процент реальных кадров: {real_percentage:.2f}%")
    print(f"Class Counts: {class_counts}")

prediction_video("test_data/test.mp4", model_)

Процент фейковых кадров: 83.67%
Процент реальных кадров: 16.33%
Class Counts: {'fake': 251, 'real': 49}
